<a href="https://colab.research.google.com/github/wwinski/cancer-classification-proj/blob/master/Cancer_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The contents of this notebook are based on a project from my ML class. The main purpose of this project is to train a model to predict types of tumors, but it also highlights the importance of preprocessing your data. Below you will see the outcome of training models on raw data, even when attempting to optimize hyperparameters.


---



## Import libraries and data from google drive

In [1]:
import pandas as pd
import numpy as np
import warnings
import itertools
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import models
from keras import layers

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
data = pd.read_csv("gdrive/My Drive/Datasets/Cancer Classification Data.csv")
labels = pd.read_csv("gdrive/My Drive/Datasets/Cancer Classification Labels.csv")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive




---


Data can be found [here](https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq#)


---



In [0]:
#prevents warning from prinitng
def warn(*args, **kwargs):
    pass

warnings.warn = warn

## View the training dataset to see what we're working with

In [4]:
data_df = data.rename(columns={'Unnamed: 0': 'Sample'}).set_index('Sample')
data_labels = labels.rename(columns={'Unnamed: 0': 'Sample'}).set_index('Sample')
data_df.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,gene_10,gene_11,gene_12,gene_13,gene_14,gene_15,gene_16,gene_17,gene_18,gene_19,gene_20,gene_21,gene_22,gene_23,gene_24,gene_25,gene_26,gene_27,gene_28,gene_29,gene_30,gene_31,gene_32,gene_33,gene_34,gene_35,gene_36,gene_37,gene_38,gene_39,...,gene_20491,gene_20492,gene_20493,gene_20494,gene_20495,gene_20496,gene_20497,gene_20498,gene_20499,gene_20500,gene_20501,gene_20502,gene_20503,gene_20504,gene_20505,gene_20506,gene_20507,gene_20508,gene_20509,gene_20510,gene_20511,gene_20512,gene_20513,gene_20514,gene_20515,gene_20516,gene_20517,gene_20518,gene_20519,gene_20520,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
Sample,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,0.591871,1.334282,2.015391,0.591871,0.0,0.0,0.0,0.0,0.591871,5.619994,1.334282,0.000000,9.796088,0.0,0.000000,1.598651,7.215116,10.839070,6.620204,9.513538,0.0,4.063658,7.764805,4.747656,13.714396,10.034496,0.000000,0.0,9.833458,0.000000,...,9.370304,10.362393,5.589928,8.141964,0.000000,2.736583,7.037152,7.123480,10.967399,5.902800,3.719370,7.203554,6.042557,2.602077,7.425526,7.846957,2.824951,6.239396,0.000000,8.469593,0.0,6.535978,6.968701,7.128881,7.175175,9.249369,7.025970,8.045563,7.475709,7.205236,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,0.000000,0.587845,2.466601,1.004394,0.0,0.0,0.0,0.0,0.000000,11.055208,3.562621,0.000000,10.070470,0.0,0.000000,0.000000,9.949812,8.522476,1.174790,4.926991,0.0,0.000000,5.819832,1.327170,13.286240,6.663316,0.587845,0.0,9.533302,0.811142,...,8.882967,9.898199,7.069401,7.186134,0.000000,3.134993,6.648930,6.715701,9.536238,1.004394,5.555482,8.029260,6.366219,0.811142,7.991732,7.161001,0.000000,4.708877,0.811142,8.451689,0.0,7.242336,8.046284,6.047558,8.572901,7.549030,7.019935,9.458940,9.190867,10.639259,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,0.000000,0.452595,1.981122,1.074163,0.0,0.0,0.0,0.0,1.683023,8.210248,4.195285,3.660427,8.970920,0.0,0.000000,0.796598,6.096650,9.861616,7.680507,3.119439,0.0,0.452595,7.899526,0.000000,10.731098,6.967883,0.452595,0.0,9.646323,0.452595,...,10.355637,10.423274,5.170201,6.194260,0.000000,3.677147,6.271990,7.089816,9.675220,0.000000,4.224017,8.020402,6.967883,5.014445,8.400038,7.527555,0.000000,4.997902,0.796598,7.761132,0.0,6.820460,8.048983,6.661493,7.716332,6.745802,7.524667,8.602350,9.036654,10.336027,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,0.000000,0.434882,2.874246,0.000000,0.0,0.0,0.0,0.0,1.267356,8.306317,3.573556,0.000000,8.524616,0.0,0.000000,0.000000,3.913761,9.511573,6.469165,7.029895,0.0,1.267356,6.800641,7.742714,12.659474,8.299890,0.768587,0.0,9.670731,0.434882,...,10.074382,9.918261,7.117924,7.196145,0.434882,3.609755,8.896696,7.577096,10.731446,5.075383,2.175652,7.675435,6.840816,6.233192,8.899886,8.319085,1.791814,5.661134,1.464093,8.625727,0.0,7.420095,7.784746,7.613915,8.963286,7.744699,7.924997,8.981473,8.665592,9.194823,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,0.000000,1.275841,2.141204,0.000000,0.0,0.0,0.0,0.0,0.889707,10.149150,2.967630,0.000000,8.047238,0.0,1.435949,0.000000,1.942120,8.821535,5.861429,7.755709,0.0,0.649386,5.570241,2.612801,13.556734,8.004754,0.000000,0.0,9.587569,0.649386,...,10.129154,10.062303,6.911620,7.855149,0.360982,3.655810,7.255520,7.292607,10.779793,3.954001,6.991148,8.153248,7.508444,4.586531,9.152227,8.

In [0]:
data_df.shape

(801, 20531)

## One-hot encoding the class labels

In [0]:
label_mappings = {
    'PRAD': [0,0,0,0,1],
    'LUAD': [0,0,0,1,0],
    'BRCA': [0,0,1,0,0],
    'KIRC': [0,1,0,0,0],
    'COAD': [1,0,0,0,0],
}

data_labels['Class'] = [label_mappings[x] for x in data_labels.Class]
one_hot_labels = np.vstack(data_labels.Class)



---


## **Purposefully not normalizing/scaling data at this point**

---



## Seperating the data into Train, Validation, and Testing sets

In [0]:
 x_train, x_test, y_train, y_test = train_test_split(data_df, one_hot_labels, test_size=0.2, random_state=1)
 x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

In [0]:
print('x_train: ' + str(len(x_train)))
print('x_val: ' + str(len(x_val)))
print('x_test: ' + str(len(x_test)))

x_train: 512
x_val: 128
x_test: 161


## Creating a single layer NN with Keras



In [58]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_dim=x_train.shape[1]))
network.add(layers.Dense(5, activation='softmax'))

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

network.fit(x_train, y_train, epochs=5, batch_size=128)

Epoch 1/5
512/512 [==============================] - 1s 3ms/step - loss: 8.8337 - acc: 0.3652
Epoch 2/5
512/512 [==============================] - 0s 349us/step - loss: 9.6331 - acc: 0.4023
Epoch 3/5
512/512 [==============================] - 0s 357us/step - loss: 9.6331 - acc: 0.4023
Epoch 4/5
512/512 [==============================] - 0s 351us/step - loss: 9.6331 - acc: 0.4023
Epoch 5/5
512/512 [==============================] - 0s 373us/step - loss: 9.6331 - acc: 0.4023


## Let's create a function so it's easier to test NN in the future

In [0]:
def create_network(train_dat, train_labels, act_func, num_layers, neuron_count, epochs, batch_size):
  network = models.Sequential()
  network.add(layers.Dense(neuron_count, activation=act_func, input_dim=train_dat.shape[1]))
  for i in range(1, num_layers):
    network.add(layers.Dense(neuron_count, activation=act_func))
  network.add(layers.Dense(5, activation='softmax'))

  network.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

  network.fit(train_dat, train_labels, epochs=epochs, batch_size=batch_size)

## Testing more layers in the network

In [0]:
create_network(x_train, y_train, 'relu', 10, 512, 5, 128)

Epoch 1/5
512/512 [==============================] - 2s 4ms/step - loss: 7.6016 - acc: 0.3984
Epoch 2/5
512/512 [==============================] - 1s 2ms/step - loss: 9.6331 - acc: 0.4023
Epoch 3/5
512/512 [==============================] - 1s 2ms/step - loss: 9.6331 - acc: 0.4023
Epoch 4/5
512/512 [==============================] - 1s 2ms/step - loss: 9.6331 - acc: 0.4023
Epoch 5/5
512/512 [==============================] - 1s 2ms/step - loss: 9.6331 - acc: 0.4023


In [0]:
create_network(x_train, y_train, 'relu', 100, 512, 5, 128)

Epoch 1/5
512/512 [==============================] - 8s 16ms/step - loss: 3.8154 - acc: 0.3340
Epoch 2/5
512/512 [==============================] - 3s 7ms/step - loss: 9.6331 - acc: 0.4023
Epoch 3/5
512/512 [==============================] - 3s 7ms/step - loss: 9.6331 - acc: 0.4023
Epoch 4/5
512/512 [==============================] - 3s 7ms/step - loss: 9.6331 - acc: 0.4023
Epoch 5/5
512/512 [==============================] - 3s 7ms/step - loss: 9.6331 - acc: 0.4023


---

Adding more layers doesn't seem to help once we reach a certain accuracy

---



## Testing more neurons per layer

In [0]:
create_network(x_train, y_train, 'relu', 5, 2048, 5, 128)

Epoch 1/5
512/512 [==============================] - 8s 15ms/step - loss: 8.1633 - acc: 0.3555
Epoch 2/5
512/512 [==============================] - 5s 10ms/step - loss: 9.6331 - acc: 0.4023
Epoch 3/5
512/512 [==============================] - 5s 10ms/step - loss: 9.6331 - acc: 0.4023
Epoch 4/5
512/512 [==============================] - 5s 10ms/step - loss: 9.6331 - acc: 0.4023
Epoch 5/5
512/512 [==============================] - 5s 10ms/step - loss: 9.6331 - acc: 0.4023


In [0]:
create_network(x_train, y_train, 'relu', 5, 4096, 5, 128)

Epoch 1/5
512/512 [==============================] - 18s 36ms/step - loss: 10.4511 - acc: 0.1758
Epoch 2/5
512/512 [==============================] - 14s 28ms/step - loss: 13.2848 - acc: 0.1758
Epoch 3/5
512/512 [==============================] - 14s 28ms/step - loss: 13.2848 - acc: 0.1758
Epoch 4/5
512/512 [==============================] - 15s 29ms/step - loss: 13.2848 - acc: 0.1758
Epoch 5/5
512/512 [==============================] - 14s 28ms/step - loss: 13.2848 - acc: 0.1758




---

Accuracy went down when the model got too complex

---



## Check the data to see why we're stuck at 40% accuracy

In [0]:
lab_df = pd.DataFrame(y_train)
lab_df = lab_df.rename(columns = {0: 'COAD', 1: 'KIRC', 2: 'BRCA', 3: 'LUAD', 4: 'PRAD'})
lab_df.sum(axis=0) / len(lab_df.index)

COAD    0.087891
KIRC    0.181641
BRCA    0.402344
LUAD    0.175781
PRAD    0.152344
dtype: float64



---

The data has BRCA class the most (40%), so the model is always guessing that

---



## Checking the same on validation set - reveals 30% BRCA

In [0]:
lab_df = pd.DataFrame(y_val)
lab_df = lab_df.rename(columns = {0: 'COAD', 1: 'KIRC', 2: 'BRCA', 3: 'LUAD', 4: 'PRAD'})
lab_df.sum(axis=0) / len(lab_df.index)

COAD    0.125000
KIRC    0.218750
BRCA    0.304688
LUAD    0.148438
PRAD    0.203125
dtype: float64

## New function to create/train a model, uses validation set for reporting

In [0]:
#Will return the confusion matrix and classification report for the given model
def train_network(act_func, num_layers, neuron_count, epochs, batch_size, optim):
  network = models.Sequential()
  network.add(layers.Dense(neuron_count, activation=act_func, input_dim=x_train.shape[1]))
  for i in range(1, num_layers):
    network.add(layers.Dense(neuron_count, activation=act_func))
  network.add(layers.Dense(5, activation='softmax'))

  network.compile(optimizer=optim,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

  network.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), epochs=epochs, batch_size=batch_size, verbose=0)

  preds = network.predict(x_val)
  preds = np.argmax(preds, axis=1)
  actuals = np.argmax(y_val, axis=1)
  return network, confusion_matrix(actuals, preds), classification_report(actuals, preds)

## Function to train model on parameter combinations and plot outcome

In [0]:
def get_optimum(param_combs, opt_param_index, xlab):
  acc_dict = {}
  opt_list = []
  for combo in param_combs:
    netw, conf, rep = train_network(*combo)
    accuracy = float(rep.split('\n')[8].split()[1])
    opt_list.append((combo, accuracy))
    acc_dict[combo[opt_param_index]] = accuracy
    print(str(combo) + ': ' + str(accuracy))
  
  # graph_results(acc_dict, xlab) #Plots not working for some cases, removing for now
  return opt_list

In [0]:
def graph_results(acc_dict, xlab):
  tups = sorted(acc_dict.items())
  x, y = zip(*tups)
  plt.bar(x, y, align='center', alpha=0.6)
  plt.xlabel(xlab)
  plt.ylabel('Accuracy')
  plt.show()

## Searching for optimal hyperparameters - testing different parameter combinations

In [0]:
opt_dict = {}

In [0]:
#OPTIMIZER
act_func_arr = ['sigmoid']
num_layer_arr = [5]
neuron_count_arr = [1024]
epoch_arr = [25]
batch_size_arr = [128]
optim_arr = ['rmsprop', 'sgd', 'adam']
param_set = [act_func_arr, num_layer_arr, neuron_count_arr, epoch_arr, batch_size_arr, optim_arr]
param_combs = itertools.product(*param_set)

opt_dict['Optimizer'] = get_optimum(param_combs, 5, 'Optimizer')

('sigmoid', 5, 1024, 25, 128, 'rmsprop'): 0.22
('sigmoid', 5, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 128, 'adam'): 0.3


In [0]:
#ACTIVATION FUNCTION
act_func_arr = ['relu', 'sigmoid', 'tanh']
num_layer_arr = [5]
neuron_count_arr = [1024]
epoch_arr = [25]
batch_size_arr = [128]
optim_arr = ['sgd']
param_set = [act_func_arr, num_layer_arr, neuron_count_arr, epoch_arr, batch_size_arr, optim_arr]
param_combs = itertools.product(*param_set)

opt_dict['Activation Function'] = get_optimum(param_combs, 0, 'Activation Function')

('relu', 5, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 128, 'sgd'): 0.3
('tanh', 5, 1024, 25, 128, 'sgd'): 0.19


In [0]:
#HIDDEN LAYERS
act_func_arr = ['sigmoid']
num_layer_arr = [4, 6, 8, 10, 12]
neuron_count_arr = [1024]
epoch_arr = [25]
batch_size_arr = [128]
optim_arr = ['sgd']
param_set = [act_func_arr, num_layer_arr, neuron_count_arr, epoch_arr, batch_size_arr, optim_arr]
param_combs = itertools.product(*param_set)

opt_dict['Hidden Layers'] = get_optimum(param_combs, 1, 'Hidden Layers')

('sigmoid', 4, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 6, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 8, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 10, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 12, 1024, 25, 128, 'sgd'): 0.3


In [0]:
#NEURON COUNT
act_func_arr = ['sigmoid']
num_layer_arr = [5]
neuron_count_arr = [512, 1024, 2048]
epoch_arr = [25]
batch_size_arr = [128]
optim_arr = ['sgd']
param_set = [act_func_arr, num_layer_arr, neuron_count_arr, epoch_arr, batch_size_arr, optim_arr]
param_combs = itertools.product(*param_set)

opt_dict['Neuron Count'] = get_optimum(param_combs, 2, 'Neuron Count')

('sigmoid', 5, 512, 25, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 5, 2048, 25, 128, 'sgd'): 0.3


In [0]:
#EPOCH COUNT
act_func_arr = ['sigmoid']
num_layer_arr = [5]
neuron_count_arr = [1024]
epoch_arr = [5, 10, 25, 50]
batch_size_arr = [128]
optim_arr = ['sgd']
param_set = [act_func_arr, num_layer_arr, neuron_count_arr, epoch_arr, batch_size_arr, optim_arr]
param_combs = itertools.product(*param_set)

opt_dict['Epoch Count'] = get_optimum(param_combs, 3, 'Epoch Count')

('sigmoid', 5, 1024, 5, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 10, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 50, 128, 'sgd'): 0.3


In [0]:
#BATCH SIZE
act_func_arr = ['sigmoid']
num_layer_arr = [5]
neuron_count_arr = [1024]
epoch_arr = [25]
batch_size_arr = [32, 64, 128, 256]
optim_arr = ['sgd']
param_set = [act_func_arr, num_layer_arr, neuron_count_arr, epoch_arr, batch_size_arr, optim_arr]
param_combs = itertools.product(*param_set)

opt_dict['Batch Size'] = get_optimum(param_combs, 4, 'Batch Size')

('sigmoid', 5, 1024, 25, 32, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 64, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 128, 'sgd'): 0.3
('sigmoid', 5, 1024, 25, 256, 'sgd'): 0.3


## Regardless of hyperparameters, unable to do any better than guessing BRCA


---


## Scaling and Normalizing the data


---



In [0]:
scaled_data = preprocessing.StandardScaler().fit_transform(data_df)
norm_data = preprocessing.normalize(scaled_data, norm='l2')

In [0]:
 x_train, x_test, y_train, y_test = train_test_split(norm_data, one_hot_labels, test_size=0.2, random_state=1)
 x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

## Retrying a basic NN with modified data

In [46]:
model, conf_m, report = train_network('relu', 5, 512, 5, 128, 'rmsprop')
print(conf_m)
print('\n')
print(report)
print('\n')
model.summary()

[[16  0  0  0  0]
 [ 0 28  0  0  0]
 [ 0  0 39  0  0]
 [ 0  0  0 19  0]
 [ 0  0  0  0 26]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        39
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        26

    accuracy                           1.00       128
   macro avg       1.00      1.00      1.00       128
weighted avg       1.00      1.00      1.00       128



Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_140 (Dense)            (None, 512)               10512384  
_________________________________________________________________
dense_141 (Dense)            (None, 512)               262656    
_______________________________________________________________

## 100% accuracy on the validation set, time to run on the test set

In [0]:
test_predictions = model.predict(x_test)

In [0]:
pred_set = []
truth_set = []
for pred, truth in zip(test_predictions, y_test):
  pred_set.append(np.argmax(pred))
  truth_set.append(np.argmax(truth))

In [55]:
conf_m = confusion_matrix(truth_set, pred_set)
report = classification_report(truth_set, pred_set)
print(conf_m)
print('\n')
print(report)

[[17  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 55  0  0]
 [ 0  0  0 32  0]
 [ 0  0  0  0 32]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        17
           1       1.00      1.00      1.00        25
           2       1.00      1.00      1.00        55
           3       1.00      1.00      1.00        32
           4       1.00      1.00      1.00        32

    accuracy                           1.00       161
   macro avg       1.00      1.00      1.00       161
weighted avg       1.00      1.00      1.00       161



## Obtained 100% accuracy on the testing set without optimizing hyperparameters, once the data was sufficiently preprocessed.